<a href="https://colab.research.google.com/github/balajis9985/balaji.github.io/blob/main/TFIDF_Job_Matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import math
import scipy
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import nltk
from nltk.corpus import stopwords

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# Step 1: Upload files interactively
from google.colab import files
uploaded = files.upload()

# Step 2: Import pandas and load the CSV files
import pandas as pd

df = pd.read_csv("VerifiedDataset (2).csv", on_bad_lines='skip')
df2 = pd.read_csv("stackoverflow_full (2).csv", on_bad_lines='skip')
linkedin_jobs = pd.read_csv("LinkedIn Jobs-Table 1 (2).csv", on_bad_lines='skip')
geton_jobs = pd.read_csv("GetOnBoard Jobs-Table 1 (2).csv", on_bad_lines='skip')

# Step 3: Print the first few rows to verify loading
print("Verified Dataset:")
print(df.head())

print("\nUnverified Dataset:")
print(df2.head())

print("\nLinkedIn Jobs Dataset:")
print(linkedin_jobs.head())

print("\nGetOnBoard Jobs Dataset:")
print(geton_jobs.head())



Saving GetOnBoard Jobs-Table 1.csv to GetOnBoard Jobs-Table 1 (3).csv
Saving LinkedIn Jobs-Table 1.csv to LinkedIn Jobs-Table 1 (3).csv
Saving stackoverflow_full.csv to stackoverflow_full (3).csv
Saving VerifiedDataset.csv to VerifiedDataset (3).csv
Saving sim_linkedin_ve.csv to sim_linkedin_ve (3).csv
Verified Dataset:
     Age Accessibility        EdLevel          Employment Gender MentalHealth  \
0    <35            No  Undergraduate  Employed full-time    Man           No   
1  35-44           Yes         Master  Employed part-time  Woman          Yes   
2    <35            No            PhD          Unemployed    Man           No   

  MainBranch  YearsCode  YearsCodePro Country  PreviousSalary  \
0        Dev          4             0  Brazil            4944   
1     DevOps         10             8     USA           85000   
2        Dev          2             1   India           12000   

          HaveWorkedWith  ComputerSkills  Employed  
0  Python;SQL;Docker;Git              4

In [14]:
#===================Data Exploration and Cleaning====================

In [15]:
# Check if dataframes are loaded
if 'df' in globals() and 'df2' in globals():
    print("Verified dataset columns:", df.columns.tolist())
    print("Unverified dataset columns:", df2.columns.tolist())

    # Check column exists before counting NaN
    if "HaveWorkedWith" in df.columns:
        nan_count = df['HaveWorkedWith'].isna().sum()
        print(f'NaN count for verified data: {nan_count}')
    else:
        print("'HaveWorkedWith' not found in verified dataset")

    if "HaveWorkedWith" in df2.columns:
        nan_count2 = df2['HaveWorkedWith'].isna().sum()
        print(f'NaN count for unverified data: {nan_count2}')
    else:
        print("'HaveWorkedWith' not found in unverified dataset")
else:
    print("Dataframes are not loaded. Please upload the CSVs first.")


Verified dataset columns: ['Age', 'Accessibility', 'EdLevel', 'Employment', 'Gender', 'MentalHealth', 'MainBranch', 'YearsCode', 'YearsCodePro', 'Country', 'PreviousSalary', 'HaveWorkedWith', 'ComputerSkills', 'Employed']
Unverified dataset columns: ['Age', 'Accessibility', 'EdLevel', 'Employment', 'Gender', 'MentalHealth', 'MainBranch', 'YearsCode', 'YearsCodePro', 'Country', 'PreviousSalary', 'HaveWorkedWith', 'ComputerSkills', 'Employed']
NaN count for verified data: 0
NaN count for unverified data: 0


In [16]:
df = df.dropna(subset=['HaveWorkedWith'])
df2 = df2.dropna(subset=['HaveWorkedWith'])

nan_count = df["HaveWorkedWith"].isna().sum()
nan_count2 = df2["HaveWorkedWith"].isna().sum()

print(f'NaN count for verified data: {nan_count}')
print(f'NaN count for unverified data: {nan_count2}')


NaN count for verified data: 0
NaN count for unverified data: 0


In [17]:
print(linkedin_jobs.columns)
print(geton_jobs.columns)

Index(['company_name', 'title', 'description', 'formatted_work_type',
       'job_posting_url', 'skills_desc'],
      dtype='object')
Index(['Source', 'Title', 'Company', 'Description', 'Link'], dtype='object')


In [18]:
# Parsing!!!

In [19]:
#  Dataset before
linkedin_jobs.head(2)

,company_name,title,description,formatted_work_type,job_posting_url,skills_desc
0,TechCorp,Data Scientist,Looking for a Python and SQL expert,Full-time,http://linkedin.com/job1,Python;SQL
1,DataWorks,Backend Developer,Experience with Java and Spring,Full-time,http://linkedin.com/job2,Java;Spring


In [20]:
# Dataset before
geton_jobs.head(2)

,Source,Title,Company,Description,Link
0,GetOnBoard,Desarrollador de Datos,TechLatam,Buscamos experto en Python y SQL,http://getonboard.com/job1
1,GetOnBoard,Desarrollador Backend,CodeMX,Experiencia en Java y Spring,http://getonboard.com/job2


In [21]:
# stopwords english version
nltk.download('stopwords')
stop_words_en = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [22]:
# parsing for mostly english text
def parse_text_linkedin(text):
  text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text) # any joint words
  text = text.lower() # lowercase
  text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # non-ASCII characters
  text = re.sub(r'\d+', '', text)  # rid of all #
  #text = re.findall(r'\b[a-zA-Z0-9\'-]+\b', text)
  words = re.findall(r'\b[a-zA-Z\'-]+\b', text)  #  keep alphabetic characters and hyphenated words
  words = [word for word in words if word not in stop_words_en] # remove stopwords english ver
  text = ' '.join(words)
  text = re.sub(r'\s+', ' ', text).strip()
  return text

# example to see if it works on english text
text_main = "€™s orders. â€¢ 34 Read physiciansâ€™ orders, measure arterial blood gases, and review patient information to assess patientâ€™s condition. â€¢ Explain treatment procedures to patients to gain cooperation and allay fears. â€¢ Monitor patientâ€™s physiological responses to therapy such as vital signs, arterial blood gases and blood chemistry changes and consult with physician if adverse reactions occur. â€¢ Administer therapeutic gases including nitrogen, nitric oxide, heliox, etc. â€¢ Enforce safety rules and ensure careful adherence to physiciansâ€™ orders. â€¢ Maintain charts that contain patient pertinent identification and therapy information. â€¢ Inspect, clean, test, and maintain respiratory therapy equipment to ensure equipment is functioning safely and efficiently and notify manager/supervisor when repairs are necessary. â€¢"

parse_text_linkedin(text_main)

'orders read physicians orders measure arterial blood gases review patient information assess patient condition explain treatment procedures patients gain cooperation allay fears monitor patient physiological responses therapy vital signs arterial blood gases blood chemistry changes consult physician adverse reactions occur administer therapeutic gases including nitrogen nitric oxide heliox etc enforce safety rules ensure careful adherence physicians orders maintain charts contain patient pertinent identification therapy information inspect clean test maintain respiratory therapy equipment ensure equipment functioning safely efficiently notify manager supervisor repairs necessary'

In [23]:
# applying the parse_text function to the description column LINKEDIN DATASET
linkedin_jobs['description'] = linkedin_jobs['description'].apply(lambda x: parse_text_linkedin(str(x)) if x is not None else "")

# show the updated dataframe
print(linkedin_jobs[['company_name','description']])

  company_name                     description
0     TechCorp       looking python sql expert
1    DataWorks          experience java spring
2     CloudNet  aws docker kubernetes required


In [24]:
# stopwords spanish version
nltk.download('stopwords')
stop_words_spa = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
# # spanish data set parse
# def parse_text_getonjobs(text):
#   text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text) # any joint words
#   text = text.lower() # lowercase
#   text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # non-ASCII characters
#   text = re.sub(r'\d+', '', text)  # rid of all numbers
#   words = re.findall(r'\b[a-zA-Z0-9\'-]+\b', text)
#   #words = re.findall(r'\b[a-zA-Z\'-]+\b', text)  #  keep alphabetic characters and hyphenated words
#   words = [word for word in words if word not in stop_words_spa] # remove stopwords spanish ver
#   text = ' '.join(words)
#   text = re.sub(r'\s+', ' ', text).strip()
#   return text

# # example to see if it works
# text_main_s = "â€¢ to and 67 allay fears. a to gases and MISIÓN la infraestructura Cloud para CMPC corresponde el stack tecnológico diferenciador que permitirá a la compañía alcanzar las metas de Innovación de acuerdo a la estrategia que acordó el directorio. Por lo cual, el Cloud Engineer deberá administrar, gestionar y controlar los contratos de administración Cloud directos e indirectos que CMPC posea con proveedores, velando por el adecuado cumplimiento de SLA´s y asegurando que el servicio entregado sea de excelencia y calidad; Así como administrar los diferentes Cloud provider que posea CMPC, eones, yaicios gestionados con terceros"

# parse_text_getonjobs(text_main_s)




# FIXED VER
# spanish data set parse
def parse_text_getonjobs(text):
  text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text) # any joint words
  text = text.lower() # lowercase
  text = re.sub(r'\d+', '', text)  # rid of all numbers

  words = re.findall(r'\b[a-záéíóúñ0-9\'-]+\b', text)  # kleep hyphenated words
  words = [word for word in words if word not in stop_words_spa] # remove spanish stop words
  text = ' '.join(words)
  text = re.sub(r'\s+', ' ', text).strip()
  return text

# example to see if it works
text_main_s = "â€¢  to and allay fears. a to od gases and MISIÓN la infraestructura Cloud para CMPC corresponde el stack tecnológico diferenciador que permitirá a la compañía alcanzar las metas de Innovación de acuerdo a la estrategia que acordó el directorio. Por lo cual, el Cloud Engineer deberá administrar, gestionar y controlar los contratos de administración Cloud directos e indirectos que CMPC posea con proveedores, velando por el adecuado cumplimiento de SLA´s y asegurando que el servicio entregado sea de excelencia y calidad; Así como administrar los diferentes Cloud provider que posea CMPC, eones, yaicios gestionados con terceros"

parse_text_getonjobs(text_main_s)

'to and allay fears to od gases and misión infraestructura cloud cmpc corresponde stack tecnológico diferenciador permitirá compañía alcanzar metas innovación acuerdo estrategia acordó directorio cloud engineer deberá administrar gestionar controlar contratos administración cloud directos indirectos cmpc posea proveedores velando adecuado cumplimiento sla s asegurando servicio entregado excelencia calidad así administrar diferentes cloud provider posea cmpc eones yaicios gestionados terceros'

In [26]:
# applying the parse_text function to the description column GETON_JOBS DATASET
geton_jobs['Description'] = geton_jobs['Description'].apply(lambda x: parse_text_getonjobs(str(x)) if x is not None else "")

# show the updated dataframe
print(geton_jobs[['Company','Description']])

      Company                     Description
0   TechLatam     buscamos experto python sql
1      CodeMX         experiencia java spring
2  CloudChile  requiere aws docker kubernetes


In [27]:
# after linkedin_jobs
linkedin_jobs.head(2)

,company_name,title,description,formatted_work_type,job_posting_url,skills_desc
0,TechCorp,Data Scientist,looking python sql expert,Full-time,http://linkedin.com/job1,Python;SQL
1,DataWorks,Backend Developer,experience java spring,Full-time,http://linkedin.com/job2,Java;Spring


In [28]:
# after geton_jobs
geton_jobs.head(2)

,Source,Title,Company,Description,Link
0,GetOnBoard,Desarrollador de Datos,TechLatam,buscamos experto python sql,http://getonboard.com/job1
1,GetOnBoard,Desarrollador Backend,CodeMX,experiencia java spring,http://getonboard.com/job2


In [29]:
#===================TF-IDF Embedding=====================

In [30]:
#TFIDF Encoding on verified dataset
vectorizer = TfidfVectorizer()

all_text = pd.concat([df["HaveWorkedWith"],df2["HaveWorkedWith"], linkedin_jobs["description"], geton_jobs["Description"]]).dropna()
vectorizer.fit(all_text)

# Transform each dataset separately using the same vectorizer
verified_tfidf = vectorizer.transform(df["HaveWorkedWith"])
unverified_tfidf = vectorizer.transform(df2["HaveWorkedWith"])

linkedin_jobs_tfidf = vectorizer.transform(linkedin_jobs["description"])
geton_jobs_tfidf = vectorizer.transform(geton_jobs["Description"])

In [31]:
#TFIDF Encoding on Jobs Datasets

# Transform each dataset separately using the same vectorizer
verified_tfidf = vectorizer.transform(df["HaveWorkedWith"])
unverified_tfidf = vectorizer.transform(df2["HaveWorkedWith"])
linkedin_jobs_tfidf = vectorizer.transform(linkedin_jobs["description"])
geton_jobs_tfidf = vectorizer.transform(geton_jobs["Description"])

In [32]:
from os import getresuid
from sklearn.metrics.pairwise import cosine_similarity

In [33]:
#NEED TO RESHAPE SINCE DATASETS ARE NOT COMPATIBLE SHAPES

#Compute cosine similarity between candidates and LinkedIn job descriptions
sim_linkedin_ver = cosine_similarity(verified_tfidf, linkedin_jobs_tfidf)
print(sim_linkedin_ver)
sim_linkedin_unver = cosine_similarity(unverified_tfidf, linkedin_jobs_tfidf)
print(sim_linkedin_unver)

[[0.33425424 0.         0.21980985]
 [0.         0.57388682 0.23346954]
 [0.16276944 0.         0.        ]]
[[0.16276944 0.         0.        ]
 [0.         0.4413982  0.        ]
 [0.17328464 0.25274785 0.25804693]]


In [34]:
# Compute cosine similarity between candidates and GetOn job descriptions
sim_geton_ver = cosine_similarity(verified_tfidf, geton_jobs_tfidf)
print(sim_geton_ver)
sim_geton_unver = cosine_similarity(unverified_tfidf, geton_jobs_tfidf)
print(sim_geton_unver)

[[0.33425424 0.         0.21980985]
 [0.         0.57388682 0.23346954]
 [0.16276944 0.         0.        ]]
[[0.16276944 0.         0.        ]
 [0.         0.4413982  0.        ]
 [0.17328464 0.25274785 0.25804693]]


In [35]:
# For each candidate, find the best match in LinkedIn
bst_mtch_linkedin_ver = sim_linkedin_ver.argmax(axis=1)
print(bst_mtch_linkedin_ver)
bst_mtch_linkedin_unver = sim_linkedin_unver.argmax(axis=1)
print(bst_mtch_linkedin_unver)

[0 1 0]
[0 1 2]


In [36]:
# For each candidate, find the best match in GetOn
bst_mtch_geton_ver = sim_geton_ver.argmax(axis=1)
print(bst_mtch_geton_ver)
bst_mtch_geton_unver = sim_geton_unver.argmax(axis=1)
print(bst_mtch_geton_unver)

[0 1 0]
[0 1 2]


In [37]:
#===================Results=====================

In [38]:
#===================LINKEDIN DATASET=======================

In [39]:
import numpy as np

most_similar_li = np.argmax(sim_linkedin_ver, axis=1)
print("Most similar LinkedIn job for each verified candidate:")
print(most_similar_li)

highest_similarity_li = np.max(sim_linkedin_ver, axis=1)
print("Highest similarity values:")
print(highest_similarity_li)

Most similar LinkedIn job for each verified candidate:
[0 1 0]
Highest similarity values:
[0.33425424 0.57388682 0.16276944]


In [40]:
print(most_similar_li)
linkedin_jobs_og = pd.read_csv("LinkedIn Jobs-Table 1.csv",on_bad_lines='skip')

most_similar_li_job_titles = df.iloc[most_similar_li]['HaveWorkedWith'].values
print(most_similar_li_job_titles)

[0 1 0]
['Python;SQL;Docker;Git' 'Java;Spring;AWS' 'Python;SQL;Docker;Git']


In [41]:
# [Verified] Display the best job for each candidate along with the similarity score

best_matches_li = np.argmax(sim_linkedin_ver, axis=1)
print(best_matches_li)

for i, match in enumerate(best_matches_li):

  if sim_linkedin_ver[i][match] > .15:
    print(f"{i}, {match}")

    print(f"Candidate {i} - {linkedin_jobs.loc[match, 'title']} with a similarity score of {sim_linkedin_ver[i][match]:.2f}")



[0 1 0]
0, 0
Candidate 0 - Data Scientist with a similarity score of 0.33
1, 1
Candidate 1 - Backend Developer with a similarity score of 0.57
2, 0
Candidate 2 - Data Scientist with a similarity score of 0.16


In [42]:
# [Unverified] Display the best job for each candidate along with the similarity score

best_matches_li_unver = np.argmax(sim_linkedin_unver, axis=1)
print(best_matches_li_unver)

for i, match in enumerate(best_matches_li_unver):

  if sim_linkedin_unver[i][match] > .10:
    print(f"{i}, {match}")

    print(f"Candidate {i} - {linkedin_jobs.loc[match, 'title']} with a similarity score of {sim_linkedin_unver[i][match]:.2f}")

[0 1 2]
0, 0
Candidate 0 - Data Scientist with a similarity score of 0.16
1, 1
Candidate 1 - Backend Developer with a similarity score of 0.44
2, 2
Candidate 2 - Cloud Engineer with a similarity score of 0.26


In [43]:
#checking if matches are the same for both unver and ver
differences_li = best_matches_li - best_matches_li_unver
differences_geton = [x for x in differences_li if x != 0]
print(differences_geton)

[np.int64(-2)]


In [44]:
#comparing candidate skills and job title match
for i, match in enumerate(best_matches_li):
  if sim_linkedin_ver[i][match] > .10:
    print(f"Candidate Skills:\t {df.loc[i, 'HaveWorkedWith']}")
    print(f"Job: \t\t\t {linkedin_jobs_og.loc[match, 'title']}")


Candidate Skills:	 Python;SQL;Docker;Git
Job: 			 Data Scientist
Candidate Skills:	 Java;Spring;AWS
Job: 			 Backend Developer
Candidate Skills:	 Python;C++;TensorFlow
Job: 			 Data Scientist


In [45]:
#===================GETON DATASET=======================

In [46]:
most_similar_geton = np.argmax(sim_geton_ver, axis=1)
print("Most similar Geton job for each verified candidate:")
print(most_similar_geton)

highest_similarity_geton = np.max(sim_geton_ver, axis=1)
print("Highest similarity values:")
print(highest_similarity_geton)

Most similar Geton job for each verified candidate:
[0 1 0]
Highest similarity values:
[0.33425424 0.57388682 0.16276944]


In [47]:
print(most_similar_geton)
geton_jobs_og = pd.read_csv("GetOnBoard Jobs-Table 1 (2).csv", on_bad_lines='skip')

most_similar_geton_job_titles = df.iloc[most_similar_geton]['HaveWorkedWith'].values
print(most_similar_geton_job_titles)

[0 1 0]
['Python;SQL;Docker;Git' 'Java;Spring;AWS' 'Python;SQL;Docker;Git']


In [48]:
# [Verified] Display the best job for each candidate along with the similarity score

best_matches_geton = np.argmax(sim_geton_ver, axis=1)
print(geton_jobs.shape)


for i, match in enumerate(best_matches_geton):
  if sim_geton_ver[i][match] > .10:
    print(f"Candidate {i} - {geton_jobs.loc[match, 'Title']} with a similarity score of {sim_geton_ver[i][match]:.2f}")


(3, 5)
Candidate 0 - Desarrollador de Datos with a similarity score of 0.33
Candidate 1 - Desarrollador Backend with a similarity score of 0.57
Candidate 2 - Desarrollador de Datos with a similarity score of 0.16


In [49]:
# [Unverified] Display the best job for each candidate along with the similarity score

best_matches_geton_unver = np.argmax(sim_geton_unver, axis=1)
print(geton_jobs.shape)

for i, match in enumerate(best_matches_geton_unver):
  if sim_geton_unver[i][match] > .10:
    print(f"Candidate {i} - {geton_jobs.loc[match, 'Title']} with a similarity score of {sim_geton_unver[i][match]:.2f}")


(3, 5)
Candidate 0 - Desarrollador de Datos with a similarity score of 0.16
Candidate 1 - Desarrollador Backend with a similarity score of 0.44
Candidate 2 - Ingeniero de Nube with a similarity score of 0.26


In [50]:
#checking if matches are the same for both unver and ver
differences_geton = best_matches_geton - best_matches_geton_unver
differences_geton = [x for x in differences_geton if x != 0]
print(differences_geton)

[np.int64(-2)]


In [51]:
# comparing candidate skills and job title match
for i, match in enumerate(best_matches_geton):
  if sim_geton_ver[i][match] > .12:
    print(f"Candidate Skills:\t {df.loc[i, 'HaveWorkedWith']}")
    print(f"Job: \t\t\t {geton_jobs.loc[match, 'Title']}")

Candidate Skills:	 Python;SQL;Docker;Git
Job: 			 Desarrollador de Datos
Candidate Skills:	 Java;Spring;AWS
Job: 			 Desarrollador Backend
Candidate Skills:	 Python;C++;TensorFlow
Job: 			 Desarrollador de Datos


In [52]:
!python --version

Python 3.11.13
